# **Activation Heat Maps in tf.Keras**

Deep learning models are black-box in nature. From a business-oriented perspective, it is important to understand how a model is making a particular prediction, more specifically -- what parts of the given input signal are contributing to a model's prediction. This is helpful for all business stakeholders.

The field of computer vision has progressed rapidly with modern deep learning cavalry. Today, it's absolutely possible to train a high-quality image classification model within a very less amount of time. But the model interpretability part still remains a major challenge. An effective deep learning practitioner should have the right tools to explain his/her deep learning models.

In this tutorial, you will explore how to visualize activation heat maps in tf.Keras.



# **Loading Cifar10 Dataset**

Firstly, you have to load our dataset.
For the purposes of this tutorial, you will use the Cifar10 Dataset.
The data is normalised before being fed into the model.

In [0]:
%tensorflow_version 2.x
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data() # Loading Data
test_images = x_test  # Set aside original validation images

x_train = np.array(x_train).astype("float32") / 255 # Normalisation
x_test = np.array(x_test).astype("float32") / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


# **Constructing the Model**

A CNN with Dropouts for this image classification task.
The dropouts are used to help avoid overfitting.
The model is compiled with RMSProp Optimizer and Categorical Crossentropy loss.


In [0]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *

inputs = Input(shape=(32, 32, 3)) # Image Shape for Cifar-10
x = (Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))(inputs)

x = (Conv2D(128, (3, 3), activation='relu', padding="same"))(x)
x = (MaxPooling2D(pool_size=(2, 2)))(x)
x = (Dropout(0.25))(x)  # Dropouts prevent overfitting

x = (Conv2D(64, (3, 3), activation='relu', padding="same"))(x)
x = (MaxPooling2D(pool_size=(2, 2)))(x)
x = (Dropout(0.25))(x)

x = (Conv2D(64, (3, 3), activation='relu', padding="same"))(x)
x = (Conv2D(32, (3, 3), activation='relu', padding="same"))(x)
x = (Dropout(0.25))(x)

x = (Conv2D(32, (3, 3), activation='relu', padding="same"))(x)
x = (Conv2D(16, (3, 3), activation='relu', padding="same"))(x)
x = (MaxPooling2D(pool_size=(2, 2)))(x)
x = (BatchNormalization())(x)

x = (Conv2D(32, (3, 3), activation='relu', padding="same"))(x)
x = (Conv2D(16, (3, 3), activation='relu', padding="same"))(x)
x = (MaxPooling2D(pool_size=(2, 2)))(x)
x = (BatchNormalization())(x)
x = (Flatten())(x)

x = (Dense(128, activation='relu'))(x)
x = (Dense(64, activation='relu'))(x)
x = (Dropout(0.5))(x)
x = (Dense(len(y_test[0]), activation='softmax'))(x)

model = Model(inputs=inputs, outputs=x)
model.compile(loss=categorical_crossentropy, optimizer=RMSprop(), metrics=['accuracy']) # Compile Model

model.fit(x_train, y_train, batch_size=512, epochs=150, verbose=1, validation_data=(x_test, y_test)) # Training

loss, accuracy = model.evaluate(x_test, y_test, verbose=0)

print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

model.save_weights("model.h5")


# **Activation Heat Maps**

Now, with the trained model, you will visualize some activation heat maps. You will use Gradient Based Class Activation Maps (Grad-CAM)

Class activation maps are a simple technique to get the discriminative image regions used by a CNN to identify a specific class in the image. In other words, a gradient based class activation map (Grad-CAM) lets you see which regions in the image were relevant to this class. This also shows how deep learning networks already have some kind of a built in attention mechanism.

Such heat maps can be useful in debugging and understanding what leads a model to make a prediction.

In such a heat map, the bluer the region, the more that region mattered to the model.


In [0]:
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import random

LAYER_NAME = 'conv2d_8' # Layer Name of Last Conv2D Layer
img_num = 4102

def get_heat_map(model, LAYER_NAME, img_data, original_image, label):
    # Create a graph that outputs target convolution and output
    grad_model = Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

    # Get the score for target class
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.array([img_data]))
        for i in range(10):
            if label[i] == 1:
                index = i
                break

        loss = predictions[:, index]

    # Extract filters and gradients
    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    # Average gradients spatially
    weights = tf.reduce_mean(grads, axis=(0, 1))

    # Build a ponderated map of filters according to gradients importance
    cam = np.ones(output.shape[0:2], dtype=np.float32)

    for index, w in enumerate(weights):
        cam += w * output[:, :, index]

    img = original_image.astype('uint8')

    # Heatmap visualization
    cam = cv2.resize(cam.numpy(), (np.shape(img)[0], np.shape(img)[1]))
    cam = np.maximum(cam, 0)
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())

    cam = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

    output_image = cv2.addWeighted(img, 0.6, cam, 0.4, 0)
    return output_image


output_image = get_heat_map(model, LAYER_NAME, x_test[img_num], test_images[img_num], y_test[img_num])
plt.subplot(1, 2, 1)
plt.imshow(test_images[img_num])

plt.subplot(1, 2, 2)
plt.imshow(output_image)

labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", 
          "frog", "horse", "ship", "truck"]

pred = model.predict(np.array([x_test[img_num]]))
max_index = 0
max_num = 0
for i in range(10):
    if pred[0][i] > max_num:
        max_num = pred[0][i]
        max_index = i

for i in range(10):
    if y_test[img_num][i] == 1:
        index = i
        break

print("The Model thinks this is a " + labels[max_index])
print("It is actually a " + labels[index])
print("Image Number:", img_num)


# **Interpreting the Results**

As you can see, the model is able to pick up on some of the defining features of a plane such as its wings.



# **Summary**

In this tutorial, you have learnt how to use tf.keras to create gradient based class activation maps (Grad-CAM) which are useful for debugging and explaining the predictions of deep learning models. Thank you!

References: https://www.sicara.ai/blog/2019-08-28-interpretability-deep-learning-tensorflow
